In [2]:
import pandas as pd
import time


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime

In [4]:
service = Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

In [5]:
url = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br'

driver.get(url)


In [6]:
download_link = driver.find_element(By.XPATH, "//a[contains(text(), 'Download')]")
download_link.click()

In [7]:
time.sleep(2)

In [8]:
download_path = 'C:/Users/rerys/Downloads/IBOVDia_10-07-24.csv'
print(f"Arquivo baixado: {download_path}")

Arquivo baixado: C:/Users/rerys/Downloads/IBOVDia_10-07-24.csv


In [15]:
df = pd.read_csv(download_path, encoding='latin1', sep=';', header=2, on_bad_lines='skip')
df = df.dropna(axis=1, how='all')




In [16]:
current_date = datetime.now().strftime('%Y-%m-%d')
df.insert(len(df.columns), '', current_date)

In [17]:
df.columns = df.columns.str.strip()
print(df.head())

   RRRP3 3R PETROLEUM  ON      NM    238.441.689  0,321            
0  ALOS3        ALLOS  ON      NM    532.616.595  0,564  2024-07-10
1  ALPA4   ALPARGATAS  PN      N1    166.362.038  0,076  2024-07-10
2  ABEV3    AMBEV S/A          ON  4.394.245.879  2,367  2024-07-10
3  ARZZ3    AREZZO CO  ON      NM     62.305.891  0,161  2024-07-10
4  ASAI3        ASSAI  ON      NM  1.349.217.892  0,742  2024-07-10
